In [2]:
import pandas as pd
import numpy as np
import re
import nltk

In [3]:
data = pd.read_csv('email_df_final.csv')
data.head()

,from,to,email,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,tana.jones@enron.com,alicia.goodrow@enron.com,nice dinner probably knowanyone else anytime w...,0,2.0,0.9333,"original_message, know, thanks, get, please, m...","['nice', 'dinner', 'probably', 'knowanyone', '..."
1,Sheila Tweed@ECT on 05/15/2001 06,Kay Mann/Corp/Enron@ENRON,absolutely good point peter start draft overri...,1,3.0,0.7429,"please, thanks, enron, need, know, deal, attac...","['absolutely', 'good', 'point', 'peter', 'star..."
2,jeff.dasovich@enron.com,christine.piesco@oracle.com,apology schedule melted talked monday swhere f...,2,2.0,0.6207,"original_message, know, thanks, get, please, m...","['apology', 'schedule', 'melted', 'talked', 'm..."
3,tanya.tamarchenko@enron.com,"Richard Lewis/LON/ECT@ECT, James New/LON/ECT@E...",vince uk var breached limit last week uk trade...,3,1.0,0.6694,"power, market, energy, state, price, californi...","['vince', 'uk', 'var', 'breached', 'limit', 'l..."
4,kay.mann@enron.com,"Don Hammond/PDX/ECT@ECT, Jody Blackburn/PDX/EC...",problem comment dale_rasmussen ectmann corp en...,4,3.0,0.6876,"please, thanks, enron, need, know, deal, attac...","['problem', 'comment', 'dale_rasmussen', 'ectm..."


In [4]:
data=data.dropna()

In [5]:
data.Dominant_Topic.unique()

array([ 2.,  3.,  1.,  8., 11., 14.,  5., 13.,  0., 10.,  4.,  9.,  7.,
        6., 12.])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9770 entries, 0 to 10347
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   from                9770 non-null   object 
 1   to                  9770 non-null   object 
 2   email               9770 non-null   object 
 3   Document_No         9770 non-null   int64  
 4   Dominant_Topic      9770 non-null   float64
 5   Topic_Perc_Contrib  9770 non-null   float64
 6   Keywords            9770 non-null   object 
 7   Text                9770 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 687.0+ KB


In [7]:
X= data.email
y=data.Dominant_Topic

In [8]:
X

0        nice dinner probably knowanyone else anytime w...
1        absolutely good point peter start draft overri...
2        apology schedule melted talked monday swhere f...
3        vince uk var breached limit last week uk trade...
4        problem comment dale_rasmussen ectmann corp en...
                               ...                        
10343    attached redline change discussed please revie...
10344    review forwarded corp enron anila hoxhaattache...
10345    hi jerry final execution version letter agreem...
10346    richard_shapiro enron com david_parquet enron ...
10347    better original_message original_message fyi d...
Name: email, Length: 9770, dtype: object

In [9]:
y.unique()

array([ 2.,  3.,  1.,  8., 11., 14.,  5., 13.,  0., 10.,  4.,  9.,  7.,
        6., 12.])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

In [11]:
len(X_train)

7816

In [12]:
len(X_test)

1954

In [13]:
len(y_train)

7816

In [14]:
len(y_test)

1954

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(stop_words='english',
                        sublinear_tf=True,
                        smooth_idf=True,
                        analyzer='word',
                        min_df=10,
                        max_df=0.95,
                        max_features=30000)
X_train = tfv.fit_transform(X_train)
X_test = tfv.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=250)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=250)

In [17]:
y_pred_rfc = rfc.predict(X_test)

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_rfc)
print(cm)
accuracy_score(y_test, y_pred_rfc)

[[  3   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
 [  5 131  15  83   0   0   0   0   0   0   1   4   0   0   1]
 [  6   3 268  78   0   0   0   0   2   0   0   4   0   0   1]
 [  9  12  34 466   0   1   0   0   1   0   0   9   0   6   1]
 [  1   0   0   0   3   0   0   0   0   0   0   0   0   0   0]
 [  3  14  10  41   0  20   0   0   0   0   0   0   0   1   1]
 [  1   1   0   0   0   0   2   0   1   0   0   5   0   0   0]
 [  1   0   2   7   0   0   0   8   1   0   0   0   0   0   0]
 [  1   6  21  29   0   0   1   0  92   0   1   5   0   0   5]
 [  1   0   0   4   0   0   0   0   5   4   0   0   0   0   0]
 [  0  10   4   8   0   0   0   0   3   0  27   1   0   0   0]
 [  0   8   7   8   0   0   0   0   0   0   0 259   0   2   0]
 [  0   1   2   2   0   0   0   0   0   0   0   0   3   0   0]
 [  1   3   0  10   0   0   0   0   0   0   0   7   1  50   0]
 [  5   2  41  13   0   0   0   0   2   0   0   2   0   1  28]]


0.6980552712384852

In [19]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf',random_state=0)
svc.fit(X_train,y_train)

SVC(random_state=0)

In [20]:
y_pred_svc = svc.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_svc)
print(cm)
accuracy_score(y_test, y_pred_svc)

[[  3   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
 [  0 161  16  57   0   0   0   0   2   0   1   2   0   0   1]
 [  0   2 311  45   0   0   0   0   1   0   0   2   0   0   1]
 [  0   8  28 487   0   0   0   0   1   0   0   6   0   9   0]
 [  0   0   0   1   3   0   0   0   0   0   0   0   0   0   0]
 [  1   9  14  45   0  19   0   0   1   0   0   0   0   1   0]
 [  0   0   1   1   0   0   3   0   2   0   0   3   0   0   0]
 [  0   0   1   9   0   0   0   8   1   0   0   0   0   0   0]
 [  0   1  14  17   0   0   0   0 120   0   1   5   0   0   3]
 [  0   0   0   5   0   0   0   0   4   5   0   0   0   0   0]
 [  0   3   4   9   0   0   0   0   3   0  34   0   0   0   0]
 [  0   8   4  12   0   0   0   0   0   0   0 256   0   2   2]
 [  0   0   3   2   0   0   0   0   0   0   0   0   3   0   0]
 [  0   0   2  14   0   0   0   0   0   0   0   3   1  52   0]
 [  0   1  38   7   0   0   0   0   3   0   0   1   0   1  43]]


0.7717502558853634